## Analyzing CIA Factbook Data Through SQL


Along the project, we'll work with the data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/). The World Factbook provides information on the history, people and society, government, economy, energy, geography, communications, transportation, military, and transnational issues for 267 world entities, i.e., a compendium of statistics about all of the countries on Earth.

In this project, we'll use SQL in Jupyter Notebook to explore and analyze data from this database. To work with the dataset locally, it can be downloaded from [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db) as `factbook.db`.

### Importing the modules and the database

In [1]:
# Code (If installing SQL from local machine for first time) # 

#!conda install -yc conda-forge ipython-sql

In [3]:
%%capture
%load_ext sql 
%sql sqlite:///factbook.db 

# Steps:
# - (capturing stdout, stderr, and IPython’s rich display() calls)
# - (loading the SQL module)
# - (connecting to a SQLite database)

'Connected: None@factbook.db'

### Overview of the data

First, we'll write a query to return information on the tables in the database. In this query, we'll filter out all tables whose names start with sqlite such as `sqlite_stat1` and `sqlite_sequence` tables. These tables are the system tables managed internally by SQLite.

After that, we'll write and run another query that returns the first rows of the table inside the database.

In [11]:
%%sql
SELECT *
    FROM sqlite_master
    WHERE type='table'
    AND name NOT LIKE 'sqlite_%';

Done.


type,name,tbl_name,rootpage,sql
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [8]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


As we can see, the database contains a table named `facts` that stores the following information about a country:

- `name` - The name of the country.
- `area` - The total land and sea area of the country in square kilometers.
- `area_land` - The country's land area in square kilometers.
- `area_water` - The country's waterarea in square kilometers.
- `population` - The country's population.
- `population_growth` - The country's population growth as a percentage.
- `birth_rate` - The country's birth rate, or the number of births a year per 1,000 people.
- `death_rate` - The country's death rate, or the number of death a year per 1,000 people.
- `migration_rate` - The difference between the number of persons entering and leaving a country during the year per 1,000 persons.

Now, let's start by calculating some summary statistics and look for any outlier countries.

### Summary statistics

First, check some general statistics from the countries:

In [13]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, MIN(population_growth) AS min_pop_growth, MAX(population_growth) AS max_pop_growth
    FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


From this results, we can extract that there is a country with a population of 0 and there is a country with a population of 7256490011, i.e., more than 7.2 billion people. Both results are unrealistic. 

Let's use subqueries to zoom in on just these countries without using the specific values.

### Checking outliers

In [14]:
%%sql
SELECT name
    FROM facts
    WHERE population IN (SELECT MIN(population)
                            FROM facts
                        );

Done.


name
Antarctica


It seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook page for [Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html).

In [15]:
%%sql
SELECT name
    FROM facts
    WHERE population IN (SELECT MAX(population)
                            FROM facts
                        );

Done.


name
World


We can also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

### Recompute summary statistics

In [16]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, MIN(population_growth) AS min_pop_growth, MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE population NOT IN (SELECT MAX(population)
                                FROM facts
                            );

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


Another option would be to use the clause `WHERE name <> 'World'`. 

Now we can see that the biggest country population is near 1.4 billion people.

Let's now calculate the average value for _population_ and _area_ columns.

### Average general population and area

In [18]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
    FROM facts
    WHERE name <> 'World';

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


### High densely populated countries

We'll build on the query we wrote for the previous screen to find countries that are densely populated. We'll identify countries that have:

- Above average values for population.
- Below average values for area.

In [23]:
%%sql
SELECT 'Country: ' || name AS high_densely_pop_countries, population, area
    FROM facts
    WHERE name <> 'World'
    AND population > (SELECT AVG(population)
                         FROM facts
                         WHERE name <> 'World'
                     )
    AND area < (SELECT AVG(area)
                         FROM facts
                         WHERE name <> 'World'
                     );

Done.


high_densely_pop_countries,population,area
Country: Bangladesh,168957745,148460
Country: Germany,80854408,357022
Country: Iraq,37056169,438317
Country: Italy,61855120,301340
Country: Japan,126919659,377915
"Country: Korea, South",49115196,99720
Country: Morocco,33322699,446550
Country: Philippines,100998376,300000
Country: Poland,38562189,312685
Country: Spain,48146134,505370


As per the results, some of the countries extracted are generally known to be densely populated. Not the same for others, but we can explain this in the fact we are taking just average values from all over the countries of the world.

Let's check now the country which has most people and the country with the highest growth rate.

### Most populated and highest growth rate countries

In [24]:
%%sql
SELECT name
    FROM facts
    WHERE population == (SELECT MAX(population)
                            FROM facts
                            WHERE name <> 'World'
                        );

Done.


name
China


In [25]:
%%sql
SELECT name
    FROM facts
    WHERE birth_rate == (SELECT MAX(birth_rate)
                            FROM facts
                            WHERE name <> 'World'
                        );

Done.


name
Niger


Following the results, _China_ and _Niger_ are the countries with the most population and the highest growth rate, respectively.

Another questions to answer could be to check the countries with the highest ratios of water to land and the countries which have more water than land.

### Highest water-to-land ratio countries and countries with more water than land area

In [28]:
%%sql
SELECT 'Country: ' || name AS high_water_to_land_ratio, CAST(area_water AS FLOAT) / CAST(area_land AS FLOAT) AS water_to_land_ratio
    FROM facts
    ORDER BY water_to_land_ratio DESC
    LIMIT 10;

Done.


high_water_to_land_ratio,water_to_land_ratio
Country: British Indian Ocean Territory,905.6666666666666
Country: Virgin Islands,4.520231213872832
Country: Puerto Rico,0.5547914317925592
"Country: Bahamas, The",0.3866133866133866
Country: Guinea-Bissau,0.2846728307254623
Country: Malawi,0.25939625850340137
Country: Netherlands,0.22571032366565366
Country: Uganda,0.22292237442922375
Country: Eritrea,0.16435643564356436
Country: Liberia,0.15623961794019933


In [29]:
%%sql
SELECT 'Country: ' || name AS more_water_than_land, area_water, area_land
    FROM facts
    WHERE area_water > area_land;

Done.


more_water_than_land,area_water,area_land
Country: British Indian Ocean Territory,54340,60
Country: Virgin Islands,1564,346


There is only two countries with more water than land area: _British Indian Ocean Territory_ and _Virgin Islands_.

We now can explore a different issue: countries which will add the most people to their population next year.

### Highest population growth countries

In [32]:
%%sql
SELECT 'Country: ' || name AS highest_population_growth, population_growth
    FROM facts
    ORDER BY population_growth DESC
    LIMIT 10;

Done.


highest_population_growth,population_growth
Country: South Sudan,4.02
Country: Malawi,3.32
Country: Burundi,3.28
Country: Niger,3.25
Country: Uganda,3.24
Country: Qatar,3.07
Country: Burkina Faso,3.03
Country: Mali,2.98
Country: Cook Islands,2.95
Country: Iraq,2.93


### Countries with the highest population decline rate

For this step, we will analyze countries with a higher death rate than birth rate and order them by their loss population ratio.

In [34]:
%%sql
SELECT 'Country: ' || name AS loss_population_countries, birth_rate, death_rate, death_rate/birth_rate AS loss_ratio
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY loss_ratio DESC;

Done.


loss_population_countries,birth_rate,death_rate,loss_ratio
Country: Bulgaria,8.92,14.44,1.6188340807174888
Country: Serbia,9.08,13.66,1.5044052863436124
Country: Latvia,10.0,14.31,1.431
Country: Lithuania,10.1,14.27,1.4128712871287128
Country: Hungary,9.16,12.73,1.3897379912663756
Country: Monaco,6.65,9.24,1.3894736842105262
Country: Slovenia,8.42,11.37,1.350356294536817
Country: Ukraine,10.72,14.46,1.3488805970149254
Country: Germany,8.47,11.42,1.3482880755608027
Country: Saint Pierre and Miquelon,7.42,9.72,1.3099730458221026


### Highest population/area ratio countries

In [46]:
%%sql
SELECT 'Country: ' || name AS highest_pop_area_ratio_countries, population, area, population/area AS pop_area_ratio
    FROM facts
    ORDER BY pop_area_ratio DESC
    LIMIT 20;

Done.


highest_pop_area_ratio_countries,population,area,pop_area_ratio
Country: Macau,592731,28,21168
Country: Monaco,30535,2,15267
Country: Singapore,5674472,697,8141
Country: Hong Kong,7141106,1108,6445
Country: Gaza Strip,1869055,360,5191
Country: Gibraltar,29258,6,4876
Country: Bahrain,1346613,760,1771
Country: Maldives,393253,298,1319
Country: Malta,413965,316,1310
Country: Bermuda,70196,54,1299


From the insgiths acquired through this new table, we can compare these results to the ones obtained before in the _High densely populated countries_ section.

We can check the countries that are common in both tables with a dynamic instruction:

In [47]:
%%sql
SELECT name
    FROM facts
    WHERE name IN (SELECT name
                    FROM facts
                    WHERE name <> 'World'
                    AND population > (SELECT AVG(population)
                         FROM facts
                         WHERE name <> 'World'
                         )
                    AND area < (SELECT AVG(area)
                         FROM facts
                         WHERE name <> 'World'
                         )
                  )
    AND name IN (SELECT name
                    FROM facts
                    ORDER BY population/area DESC
                    LIMIT 20
                );

Done.


name
Bangladesh


As we can see, only _Bangladesh_ is in the top 20 highly densely populated countries looking at its ratio. The rest of the countries we previously calculated are not inside this top.

This is the reason why it's important to take into account outliers and not to work with average values only.